# Data preprocessing notebook

## Imports

In [ ]:
import os

import pandas as pd

directory = os.getcwd().split('/')[-1]

if directory == 'notebooks':
    %cd ..

## Load the data

In [ ]:
cleaned_data_file = 'proprietary_data/cleaned_data.csv'
incidents = pd.read_csv(cleaned_data_file, encoding='utf-8', index_col=3, escapechar='\\')
incidents

## Feature Description

| Feature | Example | Note |
| :--     | :--     | :--  |
| id | 1, 2, ..., 814667 | id starting from 1 |
| hastegrad | A, H, V, V1, V2 | A=acute, H=urgent, V=Ambulance transport, V1=?, V2=? |
| tidspunkt | 13.02.2015 09:23:18 | Call received, response time starts |
| tiltak_opprettet | 13.02.2015 09:29:23 | ? |
| ressurs_id | 119 Legebil |  |
| tiltak_type | Legebil | Vehicle type |
| varslet | 13.02.2015 09:29:28 | Call answered |
| rykker_ut | 13.02.2015 09:29:33 | Dispatch time |
| ank_hentested | 10.07.2016  19:50:00  | Arrival at scene, response time stops |
| avg_hentested | 10.07.2016  20:11:00  | Scene left |
| ank_levsted | 10.07.2016  20:22:00  | Arrival at delivery location |
| ledig | 10.07.2016  20:30:00  | Time available, reset time stop |
| utrykningstid | 00:04:00 | Time from `varslet` to `ank_hentested` |
| responstid | 00:05:00 | Response time |
| gml_id | id40db6ef4-ac60-498a-ac0f-dc8104b154da | ? |
| lokalId | c4f01ec3-3922-469d-9e8c-e5c26e66c9a2 | ? |
| navnerom | http://skjema.geonorge.no/SOSI/produktspesifikasjon/BefolkningRutenett/20171201 | Grid XML specification |
| versjonId | 20171201 |  |
| oppdateringsdato | 2017-12-07T11:37:34 | Last modified |
| datauttaksdato | 2017-12-04T11:04:05 | Date of retrieval |
| opphav | Statistisk sentralbyrå | Source |
| ssbid1000M | 22640006631000 | 1x1km SSB grid id |
| rsize | 1000 | Grid size? |
| col | 223 |  |
| row | 349 |  |
| xcoor | 242500 | UTM zone-33W easting |
| ycoor | 6641500 | UTM zone-33W northing |
| popTot | 1329 | Population total in grid? |
| popAve | 38.1 | Population average in grid? |
| popFem | 692 | Female population in grid? |
| popMal | 637 | Male population in grid? |
| statistikkÅr | 2016 | Statistics year of population data? |
| geometry | c(264925, 6631996) | Centroid location? |


# Filter out irrelevant regions

In [ ]:
oslo = pd.read_csv('data/oslo.csv', encoding='utf-8', index_col=4)
akershus = pd.read_csv('data/akershus.csv', encoding='utf-8', index_col=4)
oslo_and_akershus_data = pd.concat([oslo, akershus])

# Create indices based on SSB grids
oslo_and_akershus = oslo_and_akershus_data.index
incidents_cells = pd.Index(incidents['ssbid1000M'])

# Print useful stats
print('Oslo and Akershus cell count:', oslo_and_akershus.shape[0])
print('Total cell count in dataset:', incidents_cells.unique().shape[0])
print('Cells in dataset in Oslo and Akershus:', incidents_cells.unique().isin(oslo_and_akershus).sum())
print('Empty cells in Oslo and Akershus:', oslo_and_akershus.shape[0] - incidents_cells.unique().isin(oslo_and_akershus).sum())

initial_row_count = incidents.shape[0]
print('Initial row count:', initial_row_count)

# Filter out all incident cells that are not located in Oslo and Akershus
incidents = incidents[incidents_cells.isin(oslo_and_akershus)]

# Save empty cells to file
empty_cells = oslo_and_akershus_data[~oslo_and_akershus.isin(incidents_cells)]
empty_cells.to_csv('data/empty_cells.csv')

rows_removed = initial_row_count - incidents.shape[0]
print(f'Rows removed: {rows_removed}')

## Feature dropping

The features: ... are not currently used, and can therefore be dropped.

In [ ]:
features_to_keep = ['rykker_ut', 'ank_hentested', 'avg_hentested', 'ledig', 'xcoor', 'ycoor', 'hastegrad', 'tiltak_type']
incidents = incidents[features_to_keep]
incidents

In [ ]:
incidents.groupby('hastegrad').count()

# Remove incomplete years

2001, 2002, and 2005 contains only 4 rows in total,  
while 2019 were incomplete

This could take a while (~1 min)

In [ ]:
incidents.index = pd.DatetimeIndex(incidents.index)
yearly_incidents = incidents['hastegrad'].groupby([incidents.index.year]).count()
yearly_incidents

In [ ]:
# incidents = incidents[incidents.index.year >= 2015]
# incidents = incidents[incidents.index.year < 2019]
incidents = incidents[incidents.index.year == 2018]  # <- year of interest

In [ ]:
# yearly_incidents = incidents['hastegrad'].groupby([incidents.index.year]).count()
# yearly_incidents

# Remove irrelevant dispatch type

In [ ]:
incidents = incidents[incidents['tiltak_type'] != 'Operativ Leder']

# Aggregate concurrent incidents

In [ ]:
# Incidents with this index is considered concurring incidents
index = ['tidspunkt', 'xcoor', 'ycoor']

In [ ]:
vehicles_assigned = incidents.groupby(index).size()
vehicles_assigned.name = 'vehicles_assigned'
vehicles_assigned

In [ ]:
transporting_vehicles = incidents.groupby(index).count()['avg_hentested']
transporting_vehicles.name = 'transporting_vehicles'
transporting_vehicles

In [ ]:
cancelled_vehicles = incidents.groupby(index).agg({'ank_hentested': lambda x: x.isnull().sum()})
cancelled_vehicles = cancelled_vehicles.squeeze()
cancelled_vehicles.name = 'cancelled_vehicles'
cancelled_vehicles

In [ ]:
grouped_data = pd.concat([vehicles_assigned, transporting_vehicles, cancelled_vehicles], axis=1)
incidents = incidents.set_index([incidents.index, 'xcoor', 'ycoor']).join(grouped_data)
incidents

In [ ]:
incidents.groupby(incidents.index).first()

## Save processed dataset

In [ ]:
incidents = incidents.sort_index()
incidents['rykker_ut'] = pd.to_datetime(incidents['rykker_ut'], dayfirst=True)
incidents['ank_hentested'] = pd.to_datetime(incidents['ank_hentested'], dayfirst=True)
incidents['avg_hentested'] = pd.to_datetime(incidents['avg_hentested'], dayfirst=True)
incidents['ledig'] = pd.to_datetime(incidents['ledig'], dayfirst=True)
incidents

In [ ]:
incidents.to_csv('data/processed_data.csv')